Running and Back testing the Model 

In [ ]:
import pandas as pd
from pandas import DataFrame
from sklearn.feature_extraction import DictVectorizer
from sklearn.naive_bayes import GaussianNB
import joblib
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import confusion_matrix
#Load in data
#Simpler to have no inputs here even though  technically less dyanmic 
def data_set():
    df = pd.read_csv("https://raw.githubusercontent.com/supalogix/credit_api_poc/master/cs-training.csv")
    df2 = df.drop( df.columns[0], axis=1)

    return df2
#Vectorize and get features of interest 
def features(data):
    data = data.dropna()
    data = data.to_dict(orient='records')
    vec = DictVectorizer()
    df_data = vec.fit_transform(data).toarray()
    names = vec.get_feature_names()
    df_data = DataFrame(
        df_data,
        columns=names)

    outcome_feature = df_data['SeriousDlqin2yrs']
    target_features = df_data.drop('SeriousDlqin2yrs', axis=1)

    return outcome_feature, target_features


#calling above functions
data = data_set()
outcome_feature, target_features = features(data)

X1, X2, Y1, Y2 = train_test_split(target_features, outcome_feature, test_size=0.5, random_state=0)
    
mod = GaussianNB()

#Train/Fit model
mod.fit(X1,Y1)
score = mod.score(X2, Y2)
print("% Accuracy: {0}".format(score.mean()))


#Confusion matrix to check accuracy 
confusion_output = mod.predict(X2)

confu_matrix = confusion_matrix(confusion_output, Y2)
print(confu_matrix)

% Accuracy: 0.9294088301322025
[[55798  4094]
 [  151    92]]


Deploying the Model 

In [ ]:
!pip install pomegranate

     |████████████████████████████████| 4.5MB 5.0MB/s 
  Created wheel for pomegranate: filename=pomegranate-0.13.5-cp36-cp36m-linux_x86_64.whl size=15238353 sha256=2e1d3a86f0e6b3714f5032fec0c2cd4ee8966f7b7aef89ed8aa4607ac13df2c8
  Stored in directory: /root/.cache/pip/wheels/99/d9/0a/0758302ccad41a61aa08ae4985b84d4cab5086e99c92a68560
Successfully built pomegranate


In [8]:
import pandas as pd
from pomegranate import *

df = pd.read_csv("https://raw.githubusercontent.com/supalogix/credit_api_poc/master/cs-training.csv")
df2 = df.drop( df.columns[0], axis=1)

data = df2.dropna()

data = data.to_dict(orient='records')

from sklearn.feature_extraction import DictVectorizer
from pandas import DataFrame
vec = DictVectorizer()
df_data = vec.fit_transform(data).toarray()
feature_names = vec.get_feature_names()
df_data = DataFrame(
df_data,
columns=feature_names)
outcome_feature = df_data['SeriousDlqin2yrs']
target_features = df_data.drop('SeriousDlqin2yrs', axis=1)


#from sklearn import cross_validate
from sklearn.model_selection import train_test_split
"""
    X1: independent (target) variables for first data set
    Y1: dependent (outcome) variable for first data set
    X2: independent (target) variables for the second data set
    Y2: dependent (outcome) variable for the second data set
"""

def our_model(DebtRatio, Monthlyincome, NumberOfDependents,
              NumberOfOpenCreditLinesAndLoans,
              NumberOfTime3050DaysPastDueNotWorse,
             NumberOfTime6089DaysPastDueNotWorse,
              NumberOfTimes90DaysLate,
              NumberRealEstateLoansOrLines,
              RevolvingUtilizationOfUnsecuredLines,
              age):
    
    X1, X2, Y1, Y2 = train_test_split(
    target_features, outcome_feature, test_size=0.5, random_state=0)
                            
    from sklearn.naive_bayes import GaussianNB
    mod = GaussianNB()
                          
    mod.fit(X1,Y1)
    
    new_input = target_features.iloc[0:1,:]
    new_input.iloc[:,:] = DebtRatio,Monthlyincome,NumberOfDependents,NumberOfOpenCreditLinesAndLoans,NumberOfTime3050DaysPastDueNotWorse,NumberOfTime6089DaysPastDueNotWorse,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,RevolvingUtilizationOfUnsecuredLines,age
 
    output = mod.predict(X2)
    predicted_out = mod.predict(new_input)
    
    if predicted_out == 0.0:
        #gives value to tell how much credit 
        return 'Approved for:', mod.predict_proba(new_input)[0][0]*100000
    else:
        return 'Your are considered high risk and are only be ellgible for: ',mod.predict_proba(new_input)[0][0]*100000
    
our_model(.88879,2300.0,4.0,9.0,0.0,0.0,0.0,1.0,.1999,65.0)


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


('Approved for:', 99416.95853601566)

Deployment with Flask

In [6]:
#source: https://medium.com/@kshitijvijay271199/flask-on-google-colab-f6525986797b
#other source: https://www.youtube.com/watch?v=Pc8WdnIdXZg
from flask import Flask, request
 #importing Flask to run the model on a host
#from flask_ngrok import run_with_ngrok #importing ngrok to make Flask apps available on the internet (in this case, temporarily)
#from pyngrok import ngrok
app = Flask(__name__)


#!ngrok authtoken 1hsQiSSJ05rnXkZfXKgyLkgMMbn_6Bfshx2Do22MRtsZYT9aM
#!killall ngrok
#get_ipython().system_raw('./ngrok http 5000 &')
#!ngrok http -subdomain=priceforecasting 5000 

#!ngrok by @priceforecasting
#run_with_ngrok(app)   #starts ngrok when the app is run
@app.route("/") #Assigns "/" as the URL for the home function, using Flask
def home():
  #return a simple form with a title and place for ticker and a button named "Predict"
    return '''
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<body style="background-color:lightgrey;"></body>
<div class="topnav" id="myTopnav">
  <a href="#news"></a>
  <a href="https://worlddatascience.tech" target="_blank">Home </a>
  <a href="javascript:void(0);" class="icon" onclick="myFunction()">
    <i class="fa fa-bars"></i>
  </a>
</div>
<style>
body {
  margin: 0;
  font-family: Arial, Helvetica, sans-serif;
}

.topnav {
  overflow: hidden;
  background-color: #333;
}

.topnav a {
  float: left;
  display: block;
  color: #f2f2f2;
  text-align: center;
  padding: 14px 16px;
  text-decoration: none;
  font-size: 17px;
}

.topnav a:hover {
  background-color: #ddd;
  color: black;
}

.topnav a.active {
  background-color: #04a5e3;
  color: white;
}

.topnav .icon {
  display: none;
}

@media screen and (max-width: 600px) {
  .topnav a:not(:first-child) {display: none;}
  .topnav a.icon {
    float: right;
    display: block;
  }
}

@media screen and (max-width: 600px) {
  .topnav.responsive {position: relative;}
  .topnav.responsive .icon {
    position: absolute;
    right: 0;
    top: 0;
  }
  .topnav.responsive a {
    float: none;
    display: block;
    text-align: left;
  }
}
</style>
    <center>
    <head>
    <h1> ~ Created By First Cohort ~ </h1>
    </head>
    </center>
    <center>
    <head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
  </head>
<form action='/predict' method="post" class="col s12">
      <h1>Credit risk model</h1>

<form action="/action_page_binary.asp" method="post" enctype="multipart/form-data">
  <label for="DebtRatio">What is your Debt Ratio:</label>
  <input type="text" id="DebtRatio" name="DebtRatio"><br><br>
  <label for="MonthlyIncome">Input your MonthlyIncome:</label>
  <input type="text" id="MonthlyIncome" name="MonthlyIncome"><br><br>
  <label for="NumberOfDependents">Input your amount of dependeants:</label>
  <input type="text" id="NumberOfDependents" name="NumberOfDependents"><br><br>
  <label for="NumberOfOpenCreditLinesAndLoans">Input the amount of open credit Lines & Loans:</label>
  <input type="text" id="NumberOfOpenCreditLinesAndLoans" name="NumberOfOpenCreditLinesAndLoans"><br><br>
  <label for="NumberOfTime3050DaysPastDueNotWorse">Input number of times you were 30-50 days past due:</label>
  <input type="text" id="NumberOfTime3050DaysPastDueNotWorse" name="NumberOfTime3050DaysPastDueNotWorse"><br><br>
  <label for="NumberOfTime6089DaysPastDueNotWorse">Input number of time you were 30-89 days past due:</label>
  <input type="text" id="NumberOfTime6089DaysPastDueNotWorse" name="NumberOfTime6089DaysPastDueNotWorse"><br><br>
  <label for="NumberOfTimes90DaysLate">Input number of times you were 90 days past due:</label>
  <input type="text" id="NumberOfTimes90DaysLate" name="NumberOfTimes90DaysLate"><br><br>
  <label for="NumberRealEstateLoansOrLines">Input amount fo real estate loans:</label>
  <input type="text" id="NumberRealEstateLoansOrLines" name="NumberRealEstateLoansOrLines"><br><br>
  <label for="RevolvingUtilizationOfUnsecuredLines">Input your revolving utilization of unsecured lines:</label>
  <input type="text" id="RevolvingUtilizationOfUnsecuredLines" name="RevolvingUtilizationOfUnsecuredLines"><br><br>
  <label for="age">Input your age:</label>
  <input type="text" id="age" name="age"><br><br>
  <input type="submit" value="Submit">
</form>
<form action='/predict' method="post" class="col s12">

</center>
        '''
@app.route('/predict',methods=['POST','GET']) #Assigns "/predict" as the URL for the predict function, using Flask
def predict():
    
    Debtratio = request.form["DebtRatio"]
    MonthlyIncome = request.form["MonthlyIncome"]
    NumberOfDependents = request.form['NumberOfDependents']
    NumberOfOpenCreditLinesAndLoans = request.form['NumberOfOpenCreditLinesAndLoans']
    NumberOfTime3050DaysPastDueNotWorse = request.form['NumberOfTime3050DaysPastDueNotWorse']
    NumberOfTime6089DaysPastDueNotWorse = request.form['NumberOfTime6089DaysPastDueNotWorse']
    NumberOfTimes90DaysLate = request.form['NumberOfTimes90DaysLate']
    NumberRealEstateLoansOrLines = request.form['NumberRealEstateLoansOrLines']
    RevolvingUtilizationOfUnsecuredLines = request.form['RevolvingUtilizationOfUnsecuredLines']
    age = request.form['age']
    
    
    #prediction = model.our_model()

    return '''
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/4.7.0/css/font-awesome.min.css">
<body style="background-color:lightgrey;"></body>
<div class="topnav" id="myTopnav">
  <a href="#news"></a>
  <a href="https://worlddatascience.tech" target="_blank">Home </a>
  <a href="javascript:void(0);" class="icon" onclick="myFunction()">
    <i class="fa fa-bars"></i>
  </a>
</div>
<style>
body {
  margin: 0;
  font-family: Arial, Helvetica, sans-serif;
}
.topnav {
  overflow: hidden;
  background-color: #333;
}

.topnav a {
  float: left;
  display: block;
  color: #f2f2f2;
  text-align: center;
  padding: 14px 16px;
  text-decoration: none;
  font-size: 17px;
}

.topnav a:hover {
  background-color: #ddd;
  color: black;
}

.topnav a.active {
  background-color: #04a5e3;
  color: white;
}

.topnav .icon {
  display: none;
}

@media screen and (max-width: 600px) {
  .topnav a:not(:first-child) {display: none;}
  .topnav a.icon {
    float: right;
    display: block;
  }
}

@media screen and (max-width: 600px) {
  .topnav.responsive {position: relative;}
  .topnav.responsive .icon {
    position: absolute;
    right: 0;
    top: 0;
  }
  .topnav.responsive a {
    float: none;
    display: block;
    text-align: left;
  }
}
</style>
    <center>
    <head>
    <h1> ~ Created By First Cohort ~ </h1>
    </head>
    </center>
<center>
      <head>
    <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.4.1/css/bootstrap.min.css" integrity="sha384-Vkoo8x4CGsO3+Hhxv8T/Q5PaXtkKtu6ug5TOeNV6gBiFeWPGFN9MuhOf23Q9Ifjh" crossorigin="anonymous">
  </head>
      </div>
      <br>
        ''' + "<h2> outputs: " + Debtratio + MonthlyIncome + NumberOfDependents + NumberOfOpenCreditLinesAndLoans+ NumberOfTime3050DaysPastDueNotWorse + NumberOfTime6089DaysPastDueNotWorse + NumberOfTimes90DaysLate + NumberRealEstateLoansOrLines +RevolvingUtilizationOfUnsecuredLines + age +'''
        <br>
    </div>
</center>
''' 
app.run() #run the Flask app

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
